# Data preprocessing

## Merging all indicators into one csv file for each individual

### Importing libraries

In [1]:
import pandas as pd
import glob
import csv
import math
import time
from collections import OrderedDict
import os
import time
import os.path
import warnings
warnings.filterwarnings("ignore")

In [12]:
folders = glob.glob('./Empatica E4/*')
for i in folders:
    people_days = glob.glob(f'./{i}/*')
        

### Setting static variables

In [13]:
gravX = 0
gravY = 0
gravZ = 0

EDAHertz = 4
BVPHertz = 64
TEMPHertz = 4
ACCHertz = 32

### Converting time to normal format function

In [14]:
def convertMilisToTime(milis):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(int(round(milis))))

### Processing acceleration function

In [15]:
def processAcceleration(x,y,z):
    #converting to G values: https://support.empatica.com/hc/en-us/articles/201608896-Data-export-and-formatting-from-Empatica-Connect-
    x = float(x) * 2/128
    y = float(y) * 2/128
    z = float(z) * 2/128
    #calculating effect of gravity
    alpha = 0.8
    global gravX
    global gravY
    global gravZ
    #global variables for applying low pass filter on acceleration values
    gravX = alpha * gravX + (1 - alpha) * x;
    gravY = alpha * gravY + (1 - alpha) * y;
    gravZ = alpha * gravZ + (1 - alpha) * z;
    #removing gravity's effect: https://developer.android.com/reference/android/hardware/SensorEvent.html#values
    x = x - gravX
    y = y - gravY
    z = z - gravZ
    #total acceleration from all 3 axes: http://physics.stackexchange.com/questions/41653/how-do-i-get-the-total-acceleration-from-3-axes
    overall = math.sqrt(x*x+y*y+z*z)
    return {'x':x,'y':y,'z':z,'overall':overall}

### Reading ACC data function

In [16]:
def readAccFile(file):
    dict = OrderedDict()
    #print ("-->Reading file:" + file)
    with open(file, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        i=0;
        for row in reader:
            if(i == 0):
                timestamp = float(row[0])+3600*2 #converting from string to float rounds and then to int
            elif(i == 1):    
                hertz=float(row[0])
            elif(i == 2):
                dict[timestamp]= processAcceleration(row[0],row[1],row[2])
            else:
                timestamp = timestamp + 1.0/hertz 
                dict[timestamp] = processAcceleration(row[0],row[1],row[2])
            i = i + 1
       
    return dict

### Reading HR, EDA, BVP and TEMP data function

In [17]:
def readFile(file):
    dict = OrderedDict() 
    #print ("-->Reading file:" + file)
    with open(file, 'r') as csvfile:
         reader = csv.reader(csvfile, delimiter='\n') 
         i=0;
         for row in reader:
             if(i == 0):
                 timestamp=row[0]
                 timestamp = float(timestamp)+3600*2 #converting from string to float rounds and then to int
             elif(i == 1):
                 hertz=float(row[0])
             elif(i == 2):
                 dict[timestamp]=row[0]
             else:
                 timestamp = timestamp + 1.0/hertz
                 dict[timestamp]=row[0]
             i = i + 1.0
    return dict

### Reading IBI data function

In [18]:
def readIBI_File(file):
    dict = OrderedDict()
    #print ("-->Reading file:" + file)
    with open(file, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter = ',')
        initialTimestamp = 0.0
        i = 0;
        for row in reader:
            if(i == 0):
                initialTimestamp = float(row[0])+3600*2
                #print ("Initial Timestamp ",initialTimestamp," exact Time: ",convertMilisToTime(initialTimestamp))
            else:
                timestamp = initialTimestamp + round(float(row[0]),1)
                dict[timestamp] = float(row[1])
            i = i + 1
    return dict

### Reading data and merging all data into one dataset for each individual and day

In [11]:
folders1 = glob.glob('./Empatica E4/*')

for i in folders1:
    people_days = glob.glob(f'./{i}/*')
    for day in people_days:
        participantID = day.split("/")[3]
        outputFile = day + "/mergedBioData.csv"
        ACC = {}
        ACC = readAccFile(day +'/ACC.csv')  
        HR = {}
        HR = readFile(day +'/HR.csv')
        EDA = {}
        EDA = readFile(day +'/EDA.csv')
        BVP = {}
        BVP = readFile(day +'/BVP.csv')
        TEMP = {}
        TEMP = readFile(day +'/TEMP.csv')
        try:
            IBI = {}
            IBI = readIBI_File(day +'/IBI.csv')
        except:
            print("exception", day)
            
            
        count = 0 #count of how many timestamps are the same
        total = 0 #total of measurements with the same timestamp
        start_time = convertMilisToTime(time.time()+3600*2) #calculating start time

        with open(outputFile,'w') as f1:
            writer=csv.writer(f1, delimiter=',',lineterminator='\n',)
            row ="ID","Timestamp","Hour","HRV","EDA","BVP","TEMP","ACC_X","ACC_Y","ACC_Z","ACC_Overall","SumIBI","Beats"
            writer.writerow(row)    
            for timestampHR, hr in HR.items():
                timestamp = convertMilisToTime(timestampHR)
                hour = timestamp.split(" ") #splitting timestamp and keeping hour 
                #merging HRV with EDA
                i = 0.0
                total = 0.0
                count = 0
                meanEDA = 0.0
                while i < 1.0:
                    if (timestampHR + i in EDA):
                        total = total + float(EDA[timestampHR+i])
                        count = count+1
                    i = i + 1.0/EDAHertz
                if(count > 0):
                    meanEDA = total/count
                #merging HRV with BVP 
                i = 0.0
                total = 0.0
                count = 0
                meanBVP = 0.0
                while i < 1.0:
                    if (timestampHR + i in BVP):
                        total = total + float(BVP[timestampHR+i])
                        count = count+1
                    i = i + 1.0/BVPHertz
                if(count > 0):
                    meanBVP = total/count
                #merging HRV with TEMP
                i = 0.0
                total = 0.0
                count = 0
                meanTemp = 0.0
                while i < 1.0:
                    if (timestampHR + i in TEMP):
                        total = total + float(TEMP[timestampHR+i])
                        count = count+1
                    i = i + 1.0/TEMPHertz
                if(count > 0):
                    meanTEMP = total/count
                #merging HRV with ACC
                i = 0.0
                totalX = 0.0
                totalY = 0.0
                totalZ = 0.0
                totalOverall = 0.0
                count = 0
                meanX = 0.0
                meanY = 0.0
                meanZ = 0.0
                meanOverall = 0.0
                while i < 1.0:
                    if (timestampHR + i in ACC):
                        totalX = totalX + float(ACC[timestampHR+i]['x'])
                        totalY = totalY + float(ACC[timestampHR+i]['y'])
                        totalZ = totalZ + float(ACC[timestampHR+i]['z'])
                        totalOverall = totalOverall + float(ACC[timestampHR+i]['overall'])
                        count = count+1
                    i = i + 1.0/ACCHertz
                if(count > 0):
                    meanX = totalX/count
                    meanY = totalY/count
                    meanZ = totalZ/count
                    meanOverall = totalOverall/count
                #merging HRV with IBI in 1 second timeframes: Sums up all IBI occurring in 1 sec time frame.
                i = 0.0
                total = 0.0
                count = 0
                sumIBI = 0.0
                while i < 1.0:
                    if(timestampHR + i in IBI):
                        total = total + float(IBI[timestampHR+i])
                        count = count + 1
                    i = i + 0.1
                if(count > 0):
                    sumIBI = total

                row = participantID,timestamp,hour[1],hr,meanEDA,meanBVP,meanTEMP,meanX,meanY,meanZ,meanOverall,sumIBI,count
                writer.writerow(row) 

exception ././Empatica E4/A00520 F/2307_A00520


## Merging individual datasets into one common

### Getting directories

In [19]:
directories = []
for root, dirs, files in os.walk("/Users/alexandralugova/Documents/GitHub/GEM/Empatica E4"):
    for f in files:
        if f.find("mergedBioData.csv")>=0:
            directories.append(f"{root}/{f}")

### Merging all datasets

In [22]:
df = pd.DataFrame()
for table in directories:
    df = df.append(pd.read_csv(table, parse_dates=['Timestamp']))
df = df.reset_index(drop=True)
df

,ID,Timestamp,Hour,HRV,EDA,BVP,TEMP,ACC_X,ACC_Y,ACC_Z,ACC_Overall,SumIBI,Beats
0,A00520 F,2019-07-21 01:24:45,01:24:45,61.00,1.736234,-40.107969,35.57,-0.000124,-1.126969e-03,-0.000433,0.008625,0.0,0
1,A00520 F,2019-07-21 01:24:46,01:24:46,72.00,1.730151,40.258125,35.55,0.001707,7.699929e-04,0.002845,0.011311,0.0,0
2,A00520 F,2019-07-21 01:24:47,01:24:47,81.33,1.725349,73.228437,35.55,-0.001463,-1.764202e-03,-0.002908,0.015706,0.0,0
3,A00520 F,2019-07-21 01:24:48,01:24:48,75.75,1.666123,-159.891094,35.55,-0.000630,1.896251e-04,-0.000394,0.005511,0.0,0
4,A00520 F,2019-07-21 01:24:49,01:24:49,72.40,1.685012,109.886719,35.55,0.000144,-4.122786e-04,-0.000219,0.007369,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283331,A00708,2019-07-18 04:09:30,04:09:30,151.15,0.021783,-0.140625,24.23,-0.000007,-2.220446e-16,0.000033,0.005346,0.0,0
2283332,A00708,2019-07-18 04:09:31,04:09:31,151.95,0.021462,0.160156,24.21,-0.000024,-2.220446e-16,-0.000025,0.004817,0.0,0
2283333,A00708,2019-07-18 04:09:32,04:09:32,152.73,0.022103,-0.084844,24.23,-0.000159,-2.220446e-16,0.000469,0.004291,0.0,0
2283334,A00708,2019-07-18 04:09:33,04:09:33,153.52,0.023064,0.116094,24.23,-0.000023,-2.220446e-16,0.000075,0.007182,0.0,0


## Adding additional data

### Adding tags

In [23]:
folders1 = glob.glob('./Empatica E4/*')
tags = pd.DataFrame()
for i in folders1:
    people_days = glob.glob(f'./{i}/*')
    for day in people_days:
        participantID = day.split("/")[3]
        try:
            table = pd.read_csv(day +'/tags.csv')
            table_new = []
            for i in table.iloc[:, 0]:
                i = convertMilisToTime(i)
                table_new.append(i)
            table_new = pd.DataFrame(table_new, columns = ['Timestamp'])
            table_new['ID'] = participantID
            table_new['Tag'] = 1
            tags = tags.append(table_new, ignore_index=True)
        except:
            pass

In [24]:
tags['Timestamp'] = pd.to_datetime(tags['Timestamp'])

In [25]:
df = pd.merge(df, tags, on=['Timestamp','ID'], how='left').fillna(0).drop_duplicates()

### Extracting date from datetime

In [29]:
df['Date'] = pd.to_datetime(df['Timestamp']).dt.date

### Adding working hours

In [26]:
df['Work'] = 0

In [31]:
df.loc[(df['ID'] == 'A00C5A') & (df['Timestamp'] >= '2019-07-19 08:00:00') & (df['Timestamp'] <= '2019-07-19 14:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00C5A') & (df['Timestamp'] >= '2019-07-20 08:00:00') & (df['Timestamp'] <= '2019-07-20 14:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00C5A') & (df['Timestamp'] >= '2019-07-21 08:00:00') & (df['Timestamp'] <= '2019-07-21 20:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00C5A') & (df['Timestamp'] >= '2019-07-22 14:00:00') & (df['Timestamp'] <= '2019-07-22 20:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00C5A') & (df['Timestamp'] >= '2019-07-23 20:00:00') & (df['Timestamp'] <= '2019-07-24 09:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00C5A') & (df['Timestamp'] >= '2019-07-25 08:00:00') & (df['Timestamp'] <= '2019-07-25 19:00:00'), 'Work'] = 1

df.loc[(df['ID'] == 'A00E0B') & (df['Timestamp'] >= '2019-07-17 07:00:00') & (df['Timestamp'] <= '2019-07-17 14:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00E0B') & (df['Timestamp'] >= '2019-07-18 07:00:00') & (df['Timestamp'] <= '2019-07-18 14:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00E0B') & (df['Timestamp'] >= '2019-07-19 14:00:00') & (df['Timestamp'] <= '2019-07-19 22:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00E0B') & (df['Timestamp'] >= '2019-07-20 14:00:00') & (df['Timestamp'] <= '2019-07-20 22:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00E0B') & (df['Timestamp'] >= '2019-07-21 07:00:00') & (df['Timestamp'] <= '2019-07-21 14:00:00'), 'Work'] = 1

df.loc[(df['ID'] == 'A00708') & (df['Timestamp'] >= '2019-07-16 08:00:00') & (df['Timestamp'] <= '2019-07-16 18:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00708') & (df['Timestamp'] >= '2019-07-17 08:00:00') & (df['Timestamp'] <= '2019-07-17 18:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00708') & (df['Timestamp'] >= '2019-07-18 08:00:00') & (df['Timestamp'] <= '2019-07-18 18:00:00'), 'Work'] = 1

df.loc[(df['ID'] == 'A00F3D(2)') & (df['Timestamp'] >= '2019-07-22 10:00:00') & (df['Timestamp'] <= '2019-07-22 14:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00F3D(2)') & (df['Timestamp'] >= '2019-07-23 06:00:00') & (df['Timestamp'] <= '2019-07-23 14:00:00'), 'Work'] = 1
df.loc[(df['ID'] == 'A00F3D(2)') & (df['Timestamp'] >= '2019-07-24 06:00:00') & (df['Timestamp'] <= '2019-07-24 14:00:00'), 'Work'] = 1

### Adding job titles

In [33]:
df['Job'] = 'not known'
df.loc[(df['ID'] == 'A00C5A') | (df['ID'] == 'A00708'), 'Job'] = 'doctor'
df.loc[(df['ID'] == 'A01690') | (df['ID'] == 'A01690 2F') | (df['ID'] == 'A00E0B') | (df['ID'] == 'A00F3D(2)') | (df['ID'] == 'A00F3D'), 'Job'] = 'nurse'
df.loc[(df['ID'] == 'A000C8'), 'Job'] = 'oss'

## Adding sex

In [34]:
df['Sex'] = 'not known'
df.loc[(df['ID'] == 'A00C5A') | (df['ID'] == 'A01690') | (df['ID'] == 'A01690 2F') | (df['ID'] == 'A00F3D') | (df['ID'] == 'A00F3D(2)'), 'Sex'] = 'F'
df.loc[(df['ID'] == 'A000C8') | (df['ID'] == 'A00E0B') | (df['ID'] == 'A00708'), 'Sex'] = 'M'

## Adding perceived autonomy levels

In [ ]:
df['Autonomy_perceived'] = 'not known'
df.loc[(df['ID'] == 'A00C5A') | (df['ID'] == 'A01690') | (df['ID'] == 'A000C8') |  (df['ID'] == 'A00F3D(2)'), 'Autonomy_perceived'] = '8'
df.loc[(df['ID'] == 'A00F3D') |  (df['ID'] == 'A00708'), 'Autonomy_perceived'] = '10'
df.loc[(df['ID'] == 'A01690 2F'), 'Autonomy_perceived'] = '7'
df.loc[(df['ID'] == 'A00E0B'), 'Autonomy_perceived'] = '6'

## Adding perceived stress levels

In [ ]:
df['Stress_perceived'] = 'not known'
df.loc[(df['ID'] == 'A00C5A') | (df['ID'] == 'A00F3D') | (df['ID'] == 'A000C8'), 'Stress_perceived'] = '8'
df.loc[(df['ID'] == 'A00E0B') |  (df['ID'] == 'A00708') | (df['ID'] == 'A00F3D(2)'), 'Stress_perceived'] = '7'
df.loc[(df['ID'] == 'A01690'), 'Stress_perceived'] = '5'
df.loc[(df['ID'] == 'A01690 2F'), 'Stress_perceived'] = '9'

## Adding PSS stress levels

In [ ]:
df['Stress_PSS'] = 'not known'
df.loc[(df['ID'] == 'A00C5A') | (df['ID'] == 'A000C8'), 'Stress_PSS'] = '19'
df.loc[(df['ID'] == 'A01690') |  (df['ID'] == 'A01690 2F') | (df['ID'] == 'A00708'), 'Stress_PSS'] = '22'
df.loc[(df['ID'] == 'A00E0B')  |  (df['ID'] == 'A00F3D'), 'Stress_PSS'] = '21'
df.loc[(df['ID'] == 'A00F3D(2)'), 'Stress_PSS'] = '17'

## Basic cleaning

### Checking data types and correcting them

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2283336 entries, 0 to 2283336
Data columns (total 17 columns):
 #   Column       Dtype         
---  ------       -----         
 0   ID           object        
 1   Timestamp    datetime64[ns]
 2   Hour         object        
 3   HRV          float64       
 4   EDA          float64       
 5   BVP          float64       
 6   TEMP         float64       
 7   ACC_X        float64       
 8   ACC_Y        float64       
 9   ACC_Z        float64       
 10  ACC_Overall  float64       
 11  SumIBI       float64       
 12  Beats        int64         
 13  Tag          float64       
 14  Work         int64         
 15  Job          object        
 16  Date         object        
dtypes: datetime64[ns](1), float64(10), int64(2), object(4)
memory usage: 313.6+ MB


In [31]:
df['Date'] = df['Date'].astype('datetime64[ns]')
df['Tag'] = df['Tag'].astype('int')

### Checking for missing values

In [32]:
df.isnull().sum()

ID             0
Timestamp      0
Hour           0
HRV            0
EDA            0
BVP            0
TEMP           0
ACC_X          0
ACC_Y          0
ACC_Z          0
ACC_Overall    0
SumIBI         0
Beats          0
Tag            0
Work           0
Job            0
Date           0
dtype: int64

### Checking for duplicated data

In [33]:
df.duplicated().sum()

0

### Modifying IDs for better use

In [34]:
df.loc[df['ID'] == 'A01690 2F', 'ID'] = 'A016902'
df.loc[df['ID'] == 'A01690', 'ID'] = 'A016901'
df.loc[df['ID'] == 'A00F3D(2)', 'ID'] = 'A00F3D2'
df.loc[df['ID'] == 'A00F3D', 'ID'] = 'A00F3D1'

### Setting indexes

In [35]:
df = df.set_index(['ID', 'Timestamp'])

### Reordering columns

In [36]:
df = df.reindex(['Date', 'Hour', 'HRV', 'EDA', 'BVP', 'TEMP', 'ACC_X', 'ACC_Y', 'ACC_Z',
       'ACC_Overall', 'SumIBI', 'Beats', 'Tag', 'Work', 'Job'], axis=1)

### Removing SumIBI and Beats columns

In [5]:
df = df.drop(columns=['SumIBI','Beats'])

### Renaming HRV to HR

In [6]:
df = df.rename(columns={'HRV':'HR'})

## Final dataframe

In [36]:
df

Date      Hour      HR       EDA         BVP  \
ID     Timestamp                                                                
A00520 2019-07-21 01:24:45 2019-07-21  01:24:45   61.00  1.736234  -40.107969   
       2019-07-21 01:24:46 2019-07-21  01:24:46   72.00  1.730151   40.258125   
       2019-07-21 01:24:47 2019-07-21  01:24:47   81.33  1.725349   73.228437   
       2019-07-21 01:24:48 2019-07-21  01:24:48   75.75  1.666123 -159.891094   
       2019-07-21 01:24:49 2019-07-21  01:24:49   72.40  1.685012  109.886719   
...                               ...       ...     ...       ...         ...   
A00708 2019-07-18 04:09:30 2019-07-18  04:09:30  151.15  0.021783   -0.140625   
       2019-07-18 04:09:31 2019-07-18  04:09:31  151.95  0.021462    0.160156   
       2019-07-18 04:09:32 2019-07-18  04:09:32  152.73  0.022103   -0.084844   
       2019-07-18 04:09:33 2019-07-18  04:09:33  153.52  0.023064    0.116094   
       2019-07-18 04:09:34 2019-07-18  04:09:34  154.35  0.000000   -0.296667   

                             TEMP     ACC_X         ACC_Y     ACC_Z  \
ID     Timestamp                                                      
A00520 2019-07-21 01:24:45  35.57 -0.000124 -1.126969e-03 -0.000433   
       2019-07-21 01:24:46  35.55  0.001707  7.699929e-04  0.002845   
       2019-07-21 01:24:47  35.55 -0.001463 -1.764202e-03 -0.002908   
       2019-07-21 01:24:48  35.55 -0.000630  1.896251e-04 -0.000394   
       2019-07-21 01:24:49  35.55  0.000144 -4.122786e-04 -0.000219   
...                           ...       ...           ...       ...   
A00708 2019-07-18 04:09:30  24.23 -0.000007 -2.220446e-16  0.000033   
       2019-07-18 04:09:31  24.21 -0.000024 -2.220446e-16 -0.000025   
       2019-07-18 04:09:32  24.23 -0.000159 -2.220446e-16  0.000469   
       2019-07-18 04:09:33  24.23 -0.000023 -2.220446e-16  0.000075   
       2019-07-18 04:09:34  24.23  0.000562 -2.220446e-16 -0.000315   

                            ACC_Overall  Tag  Work        Job        Sex  \
ID     Timestamp                                                           
A00520 2019-07-21 01:24:45     0.008625    0     0  not known  not known   
       2019-07-21 01:24:46     0.011311    0     0  not known  not known   
       2019-07-21 01:24:47     0.015706    0     0  not known  not known   
       2019-07-21 01:24:48     0.005511    0     0  not known  not known   
       2019-07-21 01:24:49     0.007369    0     0  not known  not known   
...                                 ...  ...   ...        ...        ...   
A00708 2019-07-18 04:09:30     0.005346    0     0     doctor          M   
       2019-07-18 04:09:31     0.004817    0     0     doctor          M   
       2019-07-18 04:09:32     0.004291    0     0     doctor          M   
       2019-07-18 04:09:33     0.007182    0     0     doctor          M   
       2019-07-18 04:09:34     0.005281    0     0     doctor          M   

                           Autonomy_perceived Stress_perceived Stress_PSS  
ID     Timestamp                                                           
A00520 2019-07-21 01:24:45          not known        not known  not known  
       2019-07-21 01:24:46          not known        not known  not known  
       2019-07-21 01:24:47          not known        not known  not known  
       2019-07-21 01:24:48          not known        not known  not known  
       2019-07-21 01:24:49          not known        not known  not known  
...                                       ...              ...        ...  
A00708 2019-07-18 04:09:30                 10                7         22  
       2019-07-18 04:09:31                 10                7         22  
       2019-07-18 04:09:32                 10                7         22  
       2019-07-18 04:09:33                 10                7         22  
       2019-07-18 04:09:34                 10                7         22  

[2283336 rows x 17 columns]